In [1]:
#I want to say thank you to the entire python community, without you I would not have learned anything).
#This is my first project - the Songwriter telegram bot on NLP. The idea is very simple - the user writes
#the first few words and Songwriter continues! Spotify-million-song-dataset was taken for training.

import os, sys
#These steps are only needed if you are deploying the bot to a server and you need to specify the path to the libraries. 
#Also don't forget to install pyTelegramBotAPI in your server's terminal.
#pip3 install pyTelegramBotAPI 
#sys.path.append('/usr/local/lib/python3.10/dist-packages/tensorflow')  
#sys.path.append('/usr/local/lib/python3.10/dist-packages/keras_nlp')
#sys.path.append('/usr/local/lib/python3.10/dist-packages/numpy')
#sys.path.append('/usr/local/lib/python3.10/dist-packages/pandas')
#sys.path.append('/usr/local/lib/python3.10/dist-packages/pandas/telebot')

!pip install keras-nlp --upgrade #Kaggle does not have keras_nlp installed by default.
import tensorflow as tf #For building ML and AI models.
from tensorflow import keras #To customize your model.
from tensorflow.keras.layers import TextVectorization #To create a vector from text.
import keras_nlp
#import telebot
import numpy as np
import pandas as pd
import random

data = pd.read_csv('/kaggle/input/spotify-million-song-dataset/spotify_millsongdata.csv')
#If you deployed your bot on a server specify the path to the dataset in a similar way.
#data = pd.read_csv('/home/your_bot_name/spotify_millsongdata.csv') 
data.head()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.1/151.1 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 54.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 68.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 1.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of tensorflow-text to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 73.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.1/578.1 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.0/578.0 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.8/511.8 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [2]:
data = data.drop(['link'], axis=1)
data = data.groupby("artist").filter(lambda x: len(x) > 186)
#I take only a small part of the dataset, because it is technically limited.
#To take performers who have more than 20 songs would require 47 gigs of RAM!
data.artist.value_counts()

Donna Summer        191
Gordon Lightfoot    189
Bob Dylan           188
George Strait       188
Alabama             187
Cher                187
Loretta Lynn        187
Reba Mcentire       187
Name: artist, dtype: int64

In [3]:
data.artist.unique()

array(['Alabama', 'Bob Dylan', 'Cher', 'Donna Summer', 'George Strait',
       'Gordon Lightfoot', 'Loretta Lynn', 'Reba Mcentire'], dtype=object)

In [4]:
data.head()

,artist,song,text
361,Alabama,Calling All Angels,"Calling, calling all angels, oh I'm calling, c..."
362,Alabama,Can't Keep A Good Man Down,I thought it was forever \r\nI thought it wou...
363,Alabama,Carolina Mountain Dew,Somewhere in the mountains......... In norther...
364,Alabama,Christmas In Dixie,"By now in New York City, there's snow on the g..."
365,Alabama,Christmas In Your Arms,All my friends are asking me where I plan to s...


In [5]:
data = data.drop(['artist'], axis=1)
data.song.value_counts()

Away In A Manger                     3
In The Garden                        3
I'll Be Home For Christmas           3
I Believe                            3
All I Really Want To Do              2
                                    ..
Here I Am Again                      1
Help Me Make It Through The Night    1
Hello Darlin'                        1
Have Mercy On Me                     1
One Thin Dime                        1
Name: song, Length: 1481, dtype: int64

In [6]:
hello = data.to_string() #This step is obligatory, string is needed for further work.
guys = hello.replace('\\r\\n', '').split('.') #Remove garbage from the data.

In [7]:
len(hello)

6462469

In [8]:
text_list = list(filter(None, guys))
random.shuffle(text_list) #Let's mix our data!

In [9]:
length = len(text_list)
text_train = text_list[:int(0.7*length)] #Split it into train, valid.
text_valid = text_list[int(0.7*length):]

In [10]:
def custom_standardization(input_string):
    sentence = tf.strings.lower(input_string)
    return sentence

maxlen = 50 # You can also set calculate the longest sentence in the data

vectorize_layer = TextVectorization(
    standardize = custom_standardization,
    output_mode="int",
    output_sequence_length=maxlen + 1,
)

vectorize_layer.adapt(text_list)
vocab = vectorize_layer.get_vocabulary()

In [11]:
vocab_size = len(vocab)
vocab_size #The size of our dictionary.

14828

In [12]:
index_lookup = dict(zip(range(len(vocab)), vocab))    
index_lookup[5]

'and'

In [13]:
batch_size = 64 #You can change this setting.

train_dataset = tf.data.Dataset.from_tensor_slices(text_train)
train_dataset = train_dataset.shuffle(buffer_size=256)
train_dataset = train_dataset.batch(batch_size)

valid_dataset = tf.data.Dataset.from_tensor_slices(text_valid)
valid_dataset = valid_dataset.shuffle(buffer_size=256)
valid_dataset = valid_dataset.batch(batch_size)

In [14]:
def preprocess_text(text):
    text = tf.expand_dims(text, -1)
    tokenized_sentences = vectorize_layer(text)
    x = tokenized_sentences[:, :-1]
    y = tokenized_sentences[:, 1:]
    return x, y


train_dataset = train_dataset.map(preprocess_text)
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)

valid_dataset = valid_dataset.map(preprocess_text)
valid_dataset = valid_dataset.prefetch(tf.data.AUTOTUNE)

In [15]:
for entry in train_dataset.take(1):
    print(entry)

(<tf.Tensor: shape=(64, 50), dtype=int64, numpy=
array([[ 116,  147,   40, ...,    0,    0,    0],
       [ 137,   65,    0, ...,    0,    0,    0],
       [  73,  484,   77, ...,    0,    0,    0],
       ...,
       [  49,   16,   10, ...,    0,    0,    0],
       [ 238,   44,   15, ..., 1791,    8,    2],
       [  61,  173,  439, ...,    0,    0,    0]])>, <tf.Tensor: shape=(64, 50), dtype=int64, numpy=
array([[ 147,   40,    5, ...,    0,    0,    0],
       [  65,    0,    0, ...,    0,    0,    0],
       [ 484,   77,  339, ...,    0,    0,    0],
       ...,
       [  16,   10, 6552, ...,    0,    0,    0],
       [  44,   15,   21, ...,    8,    2,  284],
       [ 173,  439,  140, ...,    0,    0,    0]])>)


In [16]:
embed_dim = 128
num_heads = 4

def create_model():
    inputs = keras.layers.Input(shape=(maxlen,), dtype=tf.int32)
    embedding_layer = keras_nlp.layers.TokenAndPositionEmbedding(vocab_size, maxlen, embed_dim)(inputs)
    decoder = keras_nlp.layers.TransformerDecoder(intermediate_dim=embed_dim, 
                                                            num_heads=num_heads, 
                                                            dropout=0.5)(embedding_layer)
    
    outputs = keras.layers.Dense(vocab_size, activation='softmax')(decoder)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    
    model.compile(
        optimizer="adam", 
        loss='sparse_categorical_crossentropy',
        metrics=[keras_nlp.metrics.Perplexity(), 'accuracy']
    )
    return model

model = create_model()
#This is the model we got, if resources allow - take more parameters.
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 50)]              0         
_________________________________________________________________
token_and_position_embedding (None, 50, 128)           1904384   
_________________________________________________________________
transformer_decoder (Transfo (None, 50, 128)           99584     
_________________________________________________________________
dense (Dense)                (None, 50, 14828)         1912812   
Total params: 3,916,780
Trainable params: 3,916,780
Non-trainable params: 0
_________________________________________________________________


In [17]:
class TextSampler(keras.callbacks.Callback):
    def __init__(self, start_prompt, max_tokens):
        self.start_prompt = start_prompt
        self.max_tokens = max_tokens
        
    # Helper method to choose a word from the top K probable words with respect to their probabilities
    # in a sequence
    def sample_token(self, logits):
        logits, indices = tf.math.top_k(logits, k=5, sorted=True)
        indices = np.asarray(indices).astype("int32")
        preds = keras.activations.softmax(tf.expand_dims(logits, 0))[0]
        preds = np.asarray(preds).astype("float32")
        return np.random.choice(indices, p=preds)

    def on_epoch_end(self, epoch, logs=None):
        decoded_sample = self.start_prompt
        
        for i in range(self.max_tokens-1):
            tokenized_prompt = vectorize_layer([decoded_sample])[:, :-1]
            predictions = self.model.predict([tokenized_prompt], verbose=0)
            sample_index = len(decoded_sample.strip().split())-1
            
            sampled_token = self.sample_token(predictions[0][sample_index])
            sampled_token = index_lookup[sampled_token]
            decoded_sample += " " + sampled_token
            
        print(f"\nSample text:\n{decoded_sample}...\n")

# First 5 words of a random sentence to be used as a seed
random_sentence = ' '.join(random.choice(text_valid).replace('\r\n', '').split(' ')[:4])
sampler = TextSampler(random_sentence, 30)
reducelr = keras.callbacks.ReduceLROnPlateau(patience=10, monitor='val_loss')

In [18]:
model = create_model()
history = model.fit(train_dataset, 
                    validation_data=valid_dataset,
                    batch_size=batch_size,
                    epochs=15, #You can make more epochs, but the text will become boring.
                    callbacks=[sampler, reducelr])

Epoch 1/15
38/38 [==============================] - 7s 84ms/step - loss: 6.9244 - perplexity: 1016.7596 - accuracy: 0.5293 - val_loss: 3.8851 - val_perplexity: 48.6728 - val_accuracy: 0.5820

Sample text:
  She still you i  to the you i you the to  the i you i you the i you to    you the i to you to...

Epoch 2/15
38/38 [==============================] - 3s 73ms/step - loss: 3.4325 - perplexity: 30.9531 - accuracy: 0.5928 - val_loss: 3.1376 - val_perplexity: 23.0479 - val_accuracy: 0.5820

Sample text:
  She still the  i a  a the the  and you  i and a a you and the a i and   you you you  you...

Epoch 3/15
38/38 [==============================] - 3s 72ms/step - loss: 2.8498 - perplexity: 17.2850 - accuracy: 0.5949 - val_loss: 2.8917 - val_perplexity: 18.0236 - val_accuracy: 0.5886

Sample text:
  She still to to a a and you i i i you i i a the and you i you a the a i you a i to i the a...

Epoch 4/15
38/38 [==============================] - 3s 72ms/step - loss: 2.6610 - perplexity: 14.

In [19]:
def sample_token(logits):
        logits, indices = tf.math.top_k(logits, k=5, sorted=True)
        indices = np.asarray(indices).astype("int32")
        preds = keras.activations.softmax(tf.expand_dims(logits, 0))[0]
        preds = np.asarray(preds).astype("float32")
        return np.random.choice(indices, p=preds)

def generate_text(prompt, response_length=30):
    decoded_sample = prompt
    for i in range(response_length-1):
        tokenized_prompt = vectorize_layer([decoded_sample])[:, :-1]
        predictions = model.predict([tokenized_prompt], verbose=0)
        sample_index = len(decoded_sample.strip().split())-1

        sampled_token = sample_token(predictions[0][sample_index])
        sampled_token = index_lookup[sampled_token]
        decoded_sample += " " + sampled_token
    return decoded_sample

In [20]:
generate_text('The sky is') #Let's look at the result!

"The sky is crowing late one day and the summertime,  when the world is not to come home in my face is all the blues oh i know i can't find"

In [21]:
generate_text('I love')

"I love has no one to the ground but if we gotta stay  that death is the end of the music you know the sands and he didn't i know"

In [22]:
generate_text('Can i')

"Can i can't take your side road, he rose without a fawn i know that i never had a letter at all, and it's just like to you can hardly hear"

In [23]:
#These are bot scripts - probably the simplest of all that you have seen.

#bot = telebot.TeleBot("API")

#@bot.message_handler(commands=['start'])
#def start_message(message):
    #bot.send_message(message.chat.id,'Write some words!')
    

#@bot.message_handler(content_types=['text'])
#def after_text(message):
    #j = message.text
    #bot.send_message(message.chat.id, generate_text(j))
    
#bot.infinity_polling()

#Where should I put the references? This link should definitely be here, a lot is taken from there
#https://stackabuse.com/python-for-nlp-deep-learning-text-generation-with-keras/
#Write to me in Telegram if you are interested in the project @semyonandreev.